In [196]:
import json

In [197]:
player_dict = json.load(open('processed_data/player_keys.json','r'))
player_dict[0] = "NP0"
player_dict[1] = "NP1"
player_dict[4] = "NP4"
player_dict[1823] = "NP1823"

id_to_player = {}
id_to_index = {}
index_to_player = {}
index = 0

for id_str, player in player_dict.items():
    if int(id_str) in id_to_player:
        print id_str, player, id_to_player[int(id_str)]
    id_to_player[int(id_str)] = player
    index_to_player[index] = player
    id_to_index[int(id_str)] = index
    index += 1
print len(id_to_player)
print len(id_to_index)
print len(index_to_player)

1898
1898
1898


In [198]:
import torch
import h5py
from torch.utils import data

class Dataset(data.Dataset):
    def __init__(self, filename):
        'Initialization'
        self.h5file = h5py.File('processed_data/all.h5','r')['all'][:]

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.h5file)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Load data and get label
        ids = self.h5file[index][0:10]
        l = len(id_to_index)
        X = torch.zeros(l*2)
        for i in ids[0:5]:
            X[id_to_index[i]] = 1
        for i in ids[5:10]:
            X[id_to_index[i] + l ] = 1
            
        y = torch.tensor(self.h5file[index][10],dtype=torch.float)
        
        return X, y

In [199]:
def posession_to_str(sample):
    outstr = "home"
    l = len(index_to_player)
    for idx, val in enumerate(sample[0][0:l]):
        if val == 1:
            outstr += " - " + index_to_player[idx]
    outstr += "\naway"
    for  idx, val in enumerate(sample[0][l:2*l]):
        if val == 1:
            outstr += " - " + index_to_player[idx]
    outstr += "\nscore update: " + str(int(sample[1].item()))
    return outstr

In [200]:
from torch import nn
model = nn.Linear(len(index_to_player)*2,1)
criterion = nn.MSELoss()
l_rate = 0.0001
l2_penalty = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr = l_rate,weight_decay=l2_penalty) 

In [ ]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

# Parameters
params = {'batch_size': 16,
          'shuffle': True,
          'num_workers': 6}
max_epochs = 100
train_pct = 0.9

# Datasets
full_dataset = Dataset('processed_data/all.h5')
train_size = int(train_pct * len(full_dataset))
validation_size = len(full_dataset) - train_size
train_set, validation_set = torch.utils.data.random_split(full_dataset, [train_size, validation_size])

# Generators
t_loader = data.DataLoader(train_set, **params)

v_loader = data.DataLoader(validation_set, **params)

# Loop over epochs
i = 0
j = 0
for epoch in range(max_epochs):
    # Training
    tot_loss = 0
    for batch, labels in t_loader:
        # Transfer to GPU
        batch, labels = batch.to(device), labels.to(device)
        predictions = model(batch)
        loss = criterion(predictions,labels)
        loss.backward()
        optimizer.step()
        
        tot_loss += loss
        if j > 10000:
            print "avg loss: ", tot_loss.item()/j
            print posession_to_str((batch[0],labels[0]))
            print "Predicted score update: ", predictions[0].item()
            j = 0
            tot_loss = 0
        
        j += params['batch_size']
        i += params['batch_size']

    # Validation
    j = 0
    with torch.set_grad_enabled(False):
        for batch, labels in v_loader:
            # Transfer to GPU
             # Transfer to GPU
            batch, labels = batch.to(device), labels.to(device)
            predictions = model(batch)
            loss = criterion(predictions,labels)
            tot_loss += loss
            if j > 10000:
                print "avg loss: ", tot_loss.item()/j
                print posession_to_str((batch[0],labels[0]))
                print "Predicted score update: ", predictions[0].item()
                j = 0
                tot_loss = 0

            j += params['batch_size']
            i += params['batch_size']

avg loss:  0.0937208291441
home - Steve Novak - Ricky Davis - Brian Skinner - Baron Davis - Marcus Camby
away - Michael Beasley - Joel Anthony - James Jones - Shawn Marion - Mario Chalmers
score update: 2
Predicted score update:  0.94104129076
avg loss:  0.081898351066
home - Chris Paul - Jason Smith - Trevor Ariza - David West - Willie Green
away - Monta Ellis - Andris Biedrins - Dorell Wright - Stephen Curry - David Lee
score update: 0
Predicted score update:  1.53474628925
avg loss:  0.0845194198072
home - Brian Skinner - Leandro Barbosa - Shawn Marion - Steve Nash - Amar'e Stoudemire
away - Shane Battier - Yao Ming - Chuck Hayes - Tracy McGrady - Rafer Alston
score update: 0
Predicted score update:  1.32735180855
avg loss:  0.0839271240722
home - Zach Randolph - Qyntel Woods - Ruben Patterson - Dale Davis - Wesley Person
away - Shane Battier - Bonzi Wells - Lorenzen Wright - Bo Outlaw - Earl Watson
score update: 0
Predicted score update:  0.503679871559
avg loss:  0.0879564681373
h

avg loss:  0.0764395802166
home - Anthony Randolph - Luke Ridnour - Darko Milicic - Wesley Johnson - Martell Webster
away - Marcus Thornton - Samuel Dalembert - Beno Udrih - Omri Casspi - Donte Greene
score update: 1
Predicted score update:  0.866620540619
avg loss:  0.0761493219735
home - Isaiah Rider - Roshown McLeod - Bimbo Coles - Alan Henderson - Dikembe Mutombo
away - Terry Mills - Grant Hill - Jerome Williams - Jerry Stackhouse - Lindsey Hunter
score update: 0
Predicted score update:  1.31270766258
avg loss:  0.0775359034919
home - Matt Barnes - Chris Paul - JJ Redick - DeAndre Jordan - Blake Griffin
away - Mike Conley - Zach Randolph - Marc Gasol - Tony Allen - Courtney Lee
score update: 3
Predicted score update:  0.91385859251
avg loss:  0.076195817405
home - Rick Brunson - Chris Kaman - Corey Maggette - Elton Brand - Bobby Simmons
away - Brad Miller - Mike Bibby - Peja Stojakovic - Chris Webber - Cuttino Mobley
score update: 2
Predicted score update:  0.565988302231
avg loss:

avg loss:  0.0761274758238
home - Brian Grant - Eddie Jones - Caron Butler - Lamar Odom - Rafer Alston
away - Yao Ming - Jim Jackson - Steve Francis - Kelvin Cato - Cuttino Mobley
score update: 2
Predicted score update:  0.78579056263
avg loss:  0.0755081298633
home - Speedy Claxton - Josh Smith - Joe Johnson - Shelden Williams - Zaza Pachulia
away - Quentin Richardson - Eddy Curry - Stephon Marbury - Channing Frye - Steve Francis
score update: 0
Predicted score update:  0.756740629673
avg loss:  0.0751364710994
home - Allan Houston - Charlie Ward - Larry Johnson - Kurt Thomas - Patrick Ewing
away - Horace Grant - Ike Austin - Matt Harpring - Darrell Armstrong - Anfernee Hardaway
score update: 1
Predicted score update:  0.868430018425
avg loss:  0.0762753410461
home - Dee Brown - Muggsy Bogues - Dell Curry - Kevin Willis - Tracy McGrady
away - Eddie Jones - Eddie Robinson - Anthony Mason - Baron Davis - David Wesley
score update: 2
Predicted score update:  0.955289006233
avg loss:  0.0

avg loss:  0.0756291764232
home - Dan Dickau - Lee Nailon - JR Smith - P.J. Brown - Matt Freije
away - Chris Mihm - Jumaine Jones - Kobe Bryant - Chucky Atkins - Lamar Odom
score update: 0
Predicted score update:  0.642383694649
avg loss:  0.0763578201635
home - Antonio McDyess - Rex Chapman - Clifford Robinson - Cedric Ceballos - Jason Kidd
away - Avery Johnson - David Robinson - Chuck Person - Jaren Jackson - Monty Williams
score update: 2
Predicted score update:  1.00786173344
avg loss:  0.0768214765067
home - Robin Lopez - Jared Dudley - Sebastian Telfair - Michael Redd - Markieff Morris
away - Kris Humphries - DeShawn Stevenson - Jordan Farmar - Deron Williams - Mehmet Okur
score update: 0
Predicted score update:  1.06724047661
avg loss:  0.0768389229576
home - Linas Kleiza - JR Smith - Kenyon Martin - Marcus Camby - Allen Iverson
away - Rajon Rondo - Paul Pierce - Kendrick Perkins - Ray Allen - Kevin Garnett
score update: 0
Predicted score update:  0.772735476494
avg loss:  0.075

avg loss:  0.0767787835849
home - Jason Richardson - Troy Murphy - Antawn Jamison - Erick Dampier - Gilbert Arenas
away - Courtney Alexander - Jahidi White - Richard Hamilton - Christian Laettner - Chris Whitney
score update: 2
Predicted score update:  0.949890494347
avg loss:  0.0757542143995
home - Quentin Richardson - Jamal Crawford - Eddy Curry - Zach Randolph - Stephon Marbury
away - Desmond Mason - Yi Jianlian - Mo Williams - Michael Redd - Andrew Bogut
score update: 0
Predicted score update:  0.530621528625
avg loss:  0.0764075513846
home - Joe Dumars - Grant Hill - Jerome Williams - Jerry Stackhouse - Scot Pollard
away - Rick Fox - Robert Horry - Elden Campbell - Kobe Bryant - Derek Fisher
score update: 0
Predicted score update:  1.34116756916
avg loss:  0.0765606816203
home - Brendan Haywood - Antonio Daniels - Caron Butler - Antawn Jamison - Gilbert Arenas
away - Hedo Turkoglu - Tony Battie - Pat Garrity - Jameer Nelson - Steve Francis
score update: 2
Predicted score update: 

...

avg loss:  0.0770700960494
home - Mike Bibby - Francisco Garcia - Metta World Peace - Shareef Abdur-Rahim - Corliss Williamson
away - Tim Duncan - Tony Parker - Manu Ginobili - Bruce Bowen - Fabricio Oberto
score update: 2
Predicted score update:  0.95854395628
avg loss:  0.0763539384348
home - Russell Westbrook - Jerami Grant - Andre Roberson - Steven Adams - Alex Abrines
away - Marcin Gortat - Otto Porter Jr. - Bradley Beal - Markieff Morris - John Wall
score update: 0
Predicted score update:  0.899414300919
avg loss:  0.0769727542378
home - Brian Grant - Eddie Jones - Mike James - Caron Butler - Malik Allen
away - Michael Olowokandi - Eric Piatkowski - Corey Maggette - Andre Miller - Elton Brand
score update: 3
Predicted score update:  0.812814712524
avg loss:  0.076285042321
home - Reggie Miller - Al Harrington - Jeff Foster - Jermaine O'Neal - Kenny Anderson
away - Joe Johnson - Casey Jacobsen - Stephon Marbury - Amar'e Stoudemire - Scott Williams
score update: 2
Predicted score u

avg loss:  0.0767298506472
home - Quentin Richardson - Darius Miles - Jeff McInnis - Lamar Odom - Elton Brand
away - Brent Barry - Gary Payton - Vin Baker - Rashard Lewis - Shammond Williams
score update: 2
Predicted score update:  0.910406887531
avg loss:  0.0761279816064
home - Anderson Varejao - Luol Deng - Dion Waiters - Tristan Thompson - Jarrett Jack
away - Amir Johnson - DeMar DeRozan - Terrence Ross - Chuck Hayes - Greivis Vasquez
score update: 2
Predicted score update:  1.21766829491
avg loss:  0.0769369289898
home - Michael Smith - Lee Mayberry - Blue Edwards - Shareef Abdur-Rahim - Bryant Reeves
away - Adam Keefe - Jeff Hornacek - Howard Eisley - Greg Foster - Karl Malone
score update: 0
Predicted score update:  0.926977753639
avg loss:  0.0764847593947
home - Darren Collison - Emeka Okafor - Peja Stojakovic - David West - Devin Brown
away - Rasual Butler - Al Thornton - Chris Kaman - Baron Davis - Marcus Camby
score update: 0
Predicted score update:  1.14998626709
avg loss:

avg loss:  0.0777269613248
home - Antonio McDyess - Chauncey Billups - Tayshaun Prince - Richard Hamilton - Rasheed Wallace
away - Damien Wilkins - Kevin Durant - Nick Collison - Luke Ridnour - Chris Wilcox
score update: 2
Predicted score update:  1.0255010128
avg loss:  0.0762433487767
home - Lance Stephenson - Gerald Henderson - Cody Zeller - Al Jefferson - Kemba Walker
away - Mike Conley - Zach Randolph - Marc Gasol - Vince Carter - Tony Allen
score update: 0
Predicted score update:  0.909152925014
avg loss:  0.0763541273416
home - Fred Jones - Al Harrington - Anthony Johnson - Jonathan Bender - Austin Croshere
away - Tony Battie - Dajuan Wagner - Ira Newble - Lee Nailon - Jeff McInnis
score update: 3
Predicted score update:  0.794536232948
avg loss:  0.0766431302689
home - Rick Fox - Shaquille O'Neal - Kobe Bryant - Derek Fisher - Samaki Walker
away - Jarron Collins - Andrei Kirilenko - John Stockton - Bryon Russell - Karl Malone
score update: 0
Predicted score update:  0.861020505

avg loss:  0.0769300552222
home - Darren Collison - Anthony Tolliver - Garrett Temple - Arron Afflalo - DeMarcus Cousins
away - Danilo Gallinari - Will Barton - Jameer Nelson - Darrell Arthur - Jusuf Nurkic
score update: 0
Predicted score update:  1.14539861679
avg loss:  0.0762735129164
home - Rod Strickland - Calbert Cheaney - Terry Davis - Tracy Murray - Chris Webber
away - Stephon Marbury - Chris Carr - Clifford Rozier - Tom Gugliotta - Kevin Garnett
score update: 0
Predicted score update:  0.871045291424
avg loss:  0.0761566649611
home - Eduardo Najera - Nene - Steve Blake - JR Smith - Allen Iverson
away - Paul Millsap - Ronnie Brewer - Gordan Giricek - Deron Williams - Mehmet Okur
score update: 0
Predicted score update:  0.770671725273
avg loss:  0.0768120372638
home - Nick Anderson - Derek Strong - Michael Doleac - B.J. Armstrong - Bo Outlaw
away - Grant Long - Mark West - Tyrone Corbin - Mookie Blaylock - Steven Smith
score update: 2
Predicted score update:  0.974210202694
avg 

avg loss:  0.0771491078142
home - Antonio McDyess - Richard Jefferson - Tim Duncan - Tony Parker - Manu Ginobili
away - Austin Daye - Tayshaun Prince - Greg Monroe - Richard Hamilton - Tracy McGrady
score update: 3
Predicted score update:  0.825505793095
avg loss:  0.0768098069456
home - George Lynch - Tony Massenburg - Pete Chilcutt - Lee Mayberry - Blue Edwards
away - Toni Kukoc - Jud Buechler - Randy Brown - Dennis Rodman - Jason Caffey
score update: 2
Predicted score update:  1.10614919662
avg loss:  0.0775959514581
home - Antawn Jamison - Josh Howard - Tony Delk - Steve Nash - Antoine Walker
away - Nene - Carmelo Anthony - Andre Miller - Marcus Camby - Voshon Lenard
score update: 0
Predicted score update:  1.26914381981
avg loss:  0.0768681425637
home - Josh Smith - Brandon Jennings - Rodney Stuckey - Will Bynum - Kyle Singler
away - Goran Dragic - PJ Tucker - Gerald Green - Channing Frye - Ish Smith
score update: 0
Predicted score update:  0.944646537304
avg loss:  0.077349610983

In [ ]:
print index_to_player[len(id_to_index)]
